## Info
In this notebook I've provided the primary code I used to initialize and tune my model. My workflow was not linear and so I've pieced this together as best I can, and hopefully in a way that can be reasonably understood by anyone reading it.

## Imports

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import matplotlib.pyplot as plt
from datetime import date
from fake_useragent import UserAgent
import time
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV, ElasticNetCV
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import pickle

# I built and ran the below functions direcectly in my scratch notebooks and cannot in this context import them correctly.
# All referenced functions are viewable in the included simpsons_modeling_functions.py file

import simpsons_modeling_functions

## Unpickling dataset

In [ ]:
with open("simpsons_full.pickle", "rb") as read_file:
    for_modeling = pickle.load(read_file)

## Initial attempt at model (returned a .51 r2 score)

In [3]:
# Initially did not test-train-split before scoring 

features = [col for col in for_modeling.columns if col != "Rating"]

X = for_modeling[features]
y = for_modeling["Rating"]

model = LinearRegression()

model.fit(X, y)
model.score(X, y)

KeyboardInterrupt: 

## Test-train-splitting

In [7]:
# For splitting data into a holdout group (20%) and a training set(80%), on which I performed cross-validation

features = [col for col in for_modeling.columns if col != "Rating"]

X = for_modeling[features]
y = for_modeling["Rating"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

NameError: name 'for_modeling' is not defined

In [ ]:
# Quick look at distributions of train and test sets

plt.figure(figsize=(15,6), facecolor="ghostwhite")
plt.hist(y_train, color="cornflowerblue", stacked=True)
plt.hist(y_test, color="gold", stacked=True)
plt.title("Distribution of episode ratings", fontdict={"fontsize": 20}, y=1.05);

## Continued evaluation, feature engineering and feature reduction via Lasso

In [8]:
# With the functions and code below, I made use of the following visualization methods

for_modeling.corr()

KeyboardInterrupt: 

In [ ]:
# For visualizing correlation between "Rating" and each set of features

filtered_full = pd.DataFrame(for_modeling.corrwith(for_modeling["Rating"]))
filtered_full["AV"] = abs(filtered_full[0])
filtered_full.sort_values(by="AV", ascending=False, inplace=True)
filtered_full

In [ ]:
# For visualizing the above with a heatmap

plt.figure(figsize=(15, 14))
sns.heatmap(pd.DataFrame(filtered_full["AV"]), annot = True, vmin = -1, vmax = 1, cmap = 'coolwarm');

In [9]:
# Used custom functions for scoring with 3-fold cross-validation (in modeling_functions.py)
# PolynomialFeatures as a whole brought my r2 way down, so I took to interacting features one 
# at a time as it made sense (mostly "Season Score" feature with other features)

In [ ]:
# Attempting to identify best alpha via plotting

alphalist = 10**(np.linspace(-2,2,200))
err_vec_val = np.zeros(len(alphalist))
plt.plot(np.log10(alphalist), err_vec_val);

In [6]:
# For visualizing the features included in each model iteration (refresher)

for i in len(X_train):
    print(X_train.iloc[i])

In [ ]:
# For viewing features with assigned coefficients for removal via LassoCV.
# As I changed alphas and saw new coefs drop to 0, I would remove them and 
# then re-fit and re-score the model. If r2 and MAE stayed the same or improved,
# as happened many times consecutively as I dropped to the 13 features in my final 
# model, I would run the process again.

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
lasso = LassoCV(cv=3)
lasso.fit(X_train_scaled, y_train)
lasso.score(X_train_scaled, y_train)
lasso_test_pred = lasso.predict(X_test_scaled)
r2_score(y_test, lasso_test_pred)

for i in range(len(lasso.coef_)):
    print("{} coef: {}".format(X_train.columns[i], lasso.coef_[i]))



## Final model

In [ ]:
# Setting up final model
 
features = [col for col in for_modeling.columns if col != "Rating"]

X = for_modeling[features]
y = for_modeling["Rating"]

lr_newest = LinearRegression()
lr_newest.fit(X_train, y_train)
lr_newest_test_pred = lr_newest.predict(X_test)
r2 = r2_score(y_test, lr_newest_test_pred)

In [ ]:
# Determining final values once .78 r2 score and .24 MAE cross-validated and tested with 13 features
 
final_intercept = lr_newest.intercept_
final_coefs = dict(zip(list(final_model.columns), list(lr_newest.coef_))
final_error = mae(y_test, lr_newest_test_pred)


## Looking at residuals

In [ ]:
# Reading resiudl data into a df for evaluation
 
residual_look = pd.DataFrame(X_test)
residual_look["Predicted"] = lr_newest_test_pred
residual_look["Actual"] = y_test
residual_look["Residual"] = lr_newest_test_pred - y_test
residual_look.sort_values("Residual")
residual_look.sort_values(by="Predicted", ascending=False)

In [ ]:
# Creating residual histogram

sns.set_theme(context="paper", style="whitegrid")
plt.figure(clear=True, figsize=(15,6), facecolor="white")
plt.hist(residual_look["Residual"], color="cornflowerblue");
plt.xticks(fontsize=12)
plt.savefig("Residual hist")

In [ ]:
# Plotting residuals
 
plt.figure(figsize=(15, 6))
sns.residplot(y_test_lisa, lr_newest_test_pred_lisa);
plt.savefig("Residuals")
plt.xticks(fontsize=12)
plt.xlabel("")